In [370]:
import types
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [379]:
data = pd.read_csv('C:/Users/ajcltm/PycharmProjects/BackTesting/data/stock_price_data_KOSPI.csv')
data

,Unnamed: 0,date,symbol,Adj Close,Close,High,Low,Open,Volume
0,0,2005-01-04,000020.KS,1518.722290,1748.000000,1756.000000,1722.000000,1756.000000,72000.0
1,1,2005-01-04,000040.KS,2340.000000,2340.000000,2430.000000,2310.000000,2400.000000,419300.0
2,2,2005-01-04,000050.KS,2963.146484,3734.850098,3939.389893,3734.850098,3939.389893,6600.0
3,3,2005-01-04,000060.KS,437.496582,1530.209961,1530.209961,1503.829956,1517.020020,169427.0
4,4,2005-01-04,000100.KS,10011.900391,10011.900391,10140.599609,9918.339844,9988.519531,95757.0
...,...,...,...,...,...,...,...,...,...
3039109,3039109,2021-07-16,383220.KS,525000.000000,525000.000000,547000.000000,520000.000000,542000.000000,40135.0
3039110,3039110,2021-07-16,383800.KS,10900.000000,10900.000000,10900.000000,10800.000000,10850.000000,304341.0
3039111,3039111,2021-07-16,38380K.KS,18100.000000,18100.000000,18100.000000,17950.000000,17950.000000,11323.0
3039112,3039112,2021-07-16,900140.KS,3645.000000,3645.000000,3680.000000,3620.000000,3645.000000,172550.0


In [380]:
context = types.SimpleNamespace()

In [381]:
current_time = dt.datetime(2020, 5, 25)
context.current_time = current_time

In [382]:
accounts_col = ['date', 'deposit', 'symbol', 'unitPrice', 'amounts', 'withdrawal', 'balance']
accounts_df = pd.DataFrame(columns = accounts_col)

context.accounts_df = accounts_df

context.capital_base = 50000000
print(context.accounts_df.columns)
s = pd.Series([current_time, 0, pd.NA, 0, 0, 0, context.capital_base], index=context.accounts_df.columns)
context.accounts_df = accounts_df.append(s, ignore_index=True)
context.accounts_df

Index(['date', 'deposit', 'symbol', 'unitPrice', 'amounts', 'withdrawal',
       'balance'],
      dtype='object')


,date,deposit,symbol,unitPrice,amounts,withdrawal,balance
0,2020-05-25,0,NaN,0,0,0,50000000


In [383]:
def order(context, symbols, unitPrice, amounts) :
    if isinstance(symbols, str) :
        temp_account_df = pd.DataFrame(data={'date': context.current_time, 'symbol':symbols, 'unitPrice':unitPrice, 'amounts':amounts}, index=[0])
    else :
        temp_account_df = pd.DataFrame(data={'date': context.current_time, 'symbol':symbols, 'unitPrice':unitPrice, 'amounts':amounts})
    temp_account_df=temp_account_df.assign(withdrawal=temp_account_df.unitPrice * temp_account_df.amounts)    
    
    temp_account_df = temp_account_df.assign(deposit=0)
    temp_account_df = temp_account_df.assign(balance = temp_account_df.apply(lambda x : context.accounts_df['balance'].iloc[-1] - x['withdrawal'] if x.name==0 else -x['withdrawal'], axis = 1))
    temp_account_df['balance'] = temp_account_df['balance'].cumsum()
    context.accounts_df = context.accounts_df.append(temp_account_df).reset_index(drop=True)

In [384]:
data['date'] = pd.to_datetime(data['date'])
data = data.loc[data['date']==current_time]
p1 = pd.read_csv('C:/Users/ajcltm/Desktop/p1.csv', index_col=0)
tickers = p1.tickers.apply(lambda x: '{:06d}'.format(x))

s = {'ticker': [], 'price': []}
f = {'ticker': [], 'price': []}
for ticker in tickers:
    try:
        price = data.loc[data['symbol'] == '{}.KS'.format(ticker), 'Adj Close'].iat[0]
        s['ticker'].append('{}.KS'.format(ticker))
        s['price'].append(price)
    except:
        f['ticker'].append(ticker)
        f['price'].append(0)
quantity = list(map(lambda x: int((50000000 / len(s['price'])) / x), s['price']))

order(context, s['ticker'], s['price'], quantity)
context.accounts_df

,date,deposit,symbol,unitPrice,amounts,withdrawal,balance
0,2020-05-25,0,NaN,0,0,0,50000000
1,2020-05-25,0,001230.KS,4040.0,104,420160.0,49579840.0
2,2020-05-25,0,013580.KS,16050.0,26,417300.0,49162540.0
3,2020-05-25,0,012630.KS,10200.0,41,418200.0,48744340.0
4,2020-05-25,0,004960.KS,13100.0,32,419200.0,48325140.0
...,...,...,...,...,...,...,...
114,2020-05-25,0,036460.KS,27900.0,15,418500.0,3329397.326172
115,2020-05-25,0,005880.KS,1760.0,240,422400.0,2906997.326172
116,2020-05-25,0,016740.KS,2720.0,155,421600.0,2485397.326172
117,2020-05-25,0,005820.KS,15857.099609,26,412284.589844,2073112.736328


In [385]:
def deposit(context, dollars) :
    balance = context.accounts_df['balance'].iloc[-1] + dollars
    s = pd.Series([current_time, dollars, pd.NA, 0, 0, 0, balance], index=context.accounts_df.columns)
    context.accounts_df = context.accounts_df.append(s, ignore_index=True)

In [343]:
deposit(context, 10000000)

In [386]:
context.accounts_df.loc[context.accounts_df['symbol']=='032560.KS']

,date,deposit,symbol,unitPrice,amounts,withdrawal,balance
19,2020-05-25,0,032560.KS,5249.910156,80,419992.8125,42147527.1875


In [387]:
portfolio_col = ['cash_values', 'stock_values', 'portfolio_values']
portfolio_df = pd.DataFrame(columns = portfolio_col)
portfolio_df = portfolio_df.append(pd.Series([0,0,0], index=portfolio_col), ignore_index=True)
context.portfolio_df = portfolio_df
context.portfolio_df

,cash_values,stock_values,portfolio_values
0,0,0,0


In [388]:
cash_values = context.accounts_df['balance'].iloc[-1]

In [389]:
for_symbol = context.accounts_df.loc[context.accounts_df['date']==current_time]['symbol'].dropna().unique().tolist()

In [390]:
data_temp = data.loc[data['date']==current_time]
stock_values_df = data_temp.loc[data_temp['symbol'].isin(for_symbol)][['symbol', 'Adj Close']]
stock_values_df = stock_values_df.assign(amounts = stock_values_df['symbol'].apply(lambda x : context.accounts_df.loc[context.accounts_df['symbol']==x]['amounts'].sum()))

In [391]:
stock_values_df = stock_values_df.assign(stock_values = stock_values_df['Adj Close'] * stock_values_df['amounts'])
stock_values = stock_values_df['stock_values'].sum()

In [392]:
s = pd.Series([cash_values, stock_values, cash_values+stock_values], index=context.portfolio_df.columns)
context.portfolio_df.append(s, ignore_index = True)

,cash_values,stock_values,portfolio_values
0,0,0,0
1,1650812.736328,48349187.263672,50000000.0
